In [48]:
from musixmatch import Musixmatch
import pandas as pd
import numpy as np
API_KEY = '0af4262e48632a40885bea21564b1f84'
NEW_API_KEY = '7d84cbdb5f7a89e4fd1316a29d200c4e'

musixmatch = Musixmatch(NEW_API_KEY)
data = pd.DataFrame({'id': [], 'lyrics': [], 'explicit': []})


{'message': {'header': {'status_code': 200, 'execute_time': 0.09249210357666},
  'body': {'artist_list': [{'artist': {'artist_id': 38425803,
      'artist_name': 'Teddy Swims',
      'artist_name_translation_list': [],
      'artist_comment': '',
      'artist_country': '',
      'artist_alias_list': [],
      'artist_rating': 75,
      'artist_twitter_url': '',
      'artist_credits': {'artist_list': []},
      'restricted': 0,
      'updated_time': '2022-01-31T16:06:22Z',
      'begin_date_year': '',
      'begin_date': '0000-00-00',
      'end_date_year': '',
      'end_date': '0000-00-00'}}]}}}

In [49]:
# Get most popular artists. 

def addData(dataFrame,page_num,artist_count=1,country_code='us'):
    data = dataFrame
    artists = musixmatch.chart_artists(page=page_num, page_size=artist_count, country=country_code)
    if artists['message']['header']['status_code'] == 200:
        artists = artists['message']['body']['artist_list']
        # Get their albums
        for artist in artists:
            artist_id = artist['artist']['artist_id']
            albums = musixmatch.artist_albums_get(artist_id=artist_id,g_album_name=1,page=1,page_size=100,s_release_date='asc')
            if albums['message']['header']['status_code'] == 200:
                albums = albums['message']['body']['album_list']
                # Get tracks of each album
                for album in albums:
                    album=album['album']
                    tracks = musixmatch.album_tracks_get(album_id=album['album_id'], album_mbid=album['album_mbid'],f_has_lyrics=True,page=1,page_size=100)
                    if tracks['message']['header']['status_code'] == 200:
                        tracks = tracks["message"]["body"]["track_list"]
                        # Get data for each track
                        for track in tracks:
                            track_id = track["track"]["track_id"]
                            # Get lyrics
                            track_lyrics = musixmatch.track_lyrics_get(track_id=track_id)
                            if track_lyrics['message']['header']['status_code'] == 200:
                                lyrics = track_lyrics['message']['body']['lyrics']['lyrics_body']
                                isExplicit = track_lyrics['message']['body']['lyrics']['explicit']
                                df = pd.DataFrame({'id': [track_id], 'lyrics': [lyrics], 'explicit': [isExplicit]})
                                data = pd.concat([data,df],ignore_index=True)
    return data

data = addData(data,1)
new_data = data
data

,id,lyrics,explicit
0,182983901.0,I'm going under and this time I fear there's n...,0.0
1,183343333.0,"All night, all night\nShe wants to run forever...",0.0
2,183519265.0,It's 2 AM and i'm still texting you\nAnd I can...,1.0
3,185478184.0,Turn down the lights\nTurn down the bed\nTurn ...,0.0
4,190883627.0,She's so picky\nBut she don't ever know what s...,1.0
...,...,...,...
71,258292931.0,Love can't make you stay\n\nDrunken rage came ...,0.0
72,258292932.0,I took a page out of your favorite book\nYou s...,0.0
73,258292933.0,"Goodbye, my love, you tore me up\nI heard you ...",1.0
74,258292936.0,"Yeah, I'm sorry for the way I've been actin'\n...",0.0


Get all songs for the next 11 most popular artists (will take a long time to execute)

In [72]:
for i in range(2,13):
    new_data = addData(new_data,i)
new_data

,id,lyrics,explicit
0,182983901.0,I'm going under and this time I fear there's n...,0.0
1,183343333.0,"All night, all night\nShe wants to run forever...",0.0
2,183519265.0,It's 2 AM and i'm still texting you\nAnd I can...,1.0
3,185478184.0,Turn down the lights\nTurn down the bed\nTurn ...,0.0
4,190883627.0,She's so picky\nBut she don't ever know what s...,1.0
...,...,...,...
2162,260205488.0,"I'm on an island, dirty dancing in the sun\nSo...",0.0
2163,254576974.0,She's a wonder woman\nShe knows what she likes...,0.0
2164,260205493.0,She's a wonder woman\nShe knows what she likes...,0.0
2165,254576975.0,"We were good, we were gold\nKinda dream that c...",0.0


In [71]:
data = new_data
print(data)
print(len(data))
print(len(data['id'].unique()))

               id                                             lyrics  explicit
0     182983901.0  I'm going under and this time I fear there's n...       0.0
1     183343333.0  All night, all night\nShe wants to run forever...       0.0
2     183519265.0  It's 2 AM and i'm still texting you\nAnd I can...       1.0
3     185478184.0  Turn down the lights\nTurn down the bed\nTurn ...       0.0
4     190883627.0  She's so picky\nBut she don't ever know what s...       1.0
...           ...                                                ...       ...
2162  260205488.0  I'm on an island, dirty dancing in the sun\nSo...       0.0
2163  254576974.0  She's a wonder woman\nShe knows what she likes...       0.0
2164  260205493.0  She's a wonder woman\nShe knows what she likes...       0.0
2165  254576975.0  We were good, we were gold\nKinda dream that c...       0.0
2166  260205370.0  The truth is bulletproof, there's no foolin' y...       0.0

[2167 rows x 3 columns]
2167
2158


In [76]:
# Clean the data
cleaned_data = data.drop_duplicates(['id'])
cleaned_data.dropna(inplace=True)
cleaned_data.reset_index(inplace=True)
del cleaned_data['index']
cleaned_data

C:\Users\drewz\AppData\Local\Temp\ipykernel_26700\1089091378.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data.dropna(inplace=True)


,id,lyrics,explicit
0,182983901.0,I'm going under and this time I fear there's n...,0.0
1,183343333.0,"All night, all night\nShe wants to run forever...",0.0
2,183519265.0,It's 2 AM and i'm still texting you\nAnd I can...,1.0
3,185478184.0,Turn down the lights\nTurn down the bed\nTurn ...,0.0
4,190883627.0,She's so picky\nBut she don't ever know what s...,1.0
...,...,...,...
2153,260205488.0,"I'm on an island, dirty dancing in the sun\nSo...",0.0
2154,254576974.0,She's a wonder woman\nShe knows what she likes...,0.0
2155,260205493.0,She's a wonder woman\nShe knows what she likes...,0.0
2156,254576975.0,"We were good, we were gold\nKinda dream that c...",0.0


In [82]:
# Write to .csv
cleaned_data.to_csv("data/tracks.csv")

In [83]:
# Read from .csv to ensure validity
read_data = pd.read_csv('data/tracks.csv', index_col=0)
read_data

,id,lyrics,explicit
0,182983901.0,I'm going under and this time I fear there's n...,0.0
1,183343333.0,"All night, all night\nShe wants to run forever...",0.0
2,183519265.0,It's 2 AM and i'm still texting you\nAnd I can...,1.0
3,185478184.0,Turn down the lights\nTurn down the bed\nTurn ...,0.0
4,190883627.0,She's so picky\nBut she don't ever know what s...,1.0
...,...,...,...
2153,260205488.0,"I'm on an island, dirty dancing in the sun\nSo...",0.0
2154,254576974.0,She's a wonder woman\nShe knows what she likes...,0.0
2155,260205493.0,She's a wonder woman\nShe knows what she likes...,0.0
2156,254576975.0,"We were good, we were gold\nKinda dream that c...",0.0
